In [2]:
import requests

def completions(email, password, message, model="qwen72", base_url='http://71.2.255.46:28080'):
    # 获取 token
    auth_url = f"{base_url}/api/v1/auths/signin"
    auth_data = {
        "email": email,
        "password": password
    }
    auth_response = requests.post(auth_url, json=auth_data)
    
    if auth_response.status_code != 200:
        raise Exception(f"Failed to get token: {auth_response.status_code} - {auth_response.text}")
    
    token = auth_response.json().get('token')
    
    # 发送消息
    chat_url = f"{base_url}/api/chat/completions"
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }
    chat_data = {
        "stream": False,
        "model": model,
        "messages": [
            {
                "role": "user",
                "content": message
            }
        ],
        "params": {},
        "features": {
            "image_generation": False,
            "code_interpreter": False,
            "web_search": False
        },
        "variables": {
            "{{USER_NAME}}": "lyh",
            "{{USER_LOCATION}}": "Unknown",
            "{{CURRENT_DATETIME}}": "2025-05-23 13:25:01",
            "{{CURRENT_DATE}}": "2025-05-23",
            "{{CURRENT_TIME}}": "13:25:01",
            "{{CURRENT_WEEKDAY}}": "Friday",
            "{{CURRENT_TIMEZONE}}": "Asia/Shanghai",
            "{{USER_LANGUAGE}}": "zh-CN"
        },
        "chat_id": "local",
    }
    chat_response = requests.post(chat_url, headers=headers, json=chat_data)
    
    if chat_response.status_code != 200:
        raise Exception(f"Failed to send message: {chat_response.status_code} - {chat_response.text}")
    
    return chat_response.json()['choices'][0]['message']['content']

# Example usage
email = "374656045@qq.com"
password = "hrdhreth1"
message = "你好"

try:
    response = completions(email, password, message)
    print("Response:", response)
except Exception as e:
    print(e)

Response: 你好！有什么可以帮助你的吗？


In [21]:
import requests
import json
def completions(email, password, message, model="qwen72", base_url='http://71.2.255.46:28080'):
    # 获取 token
    auth_url = f"{base_url}/api/v1/auths/signin"
    auth_data = {
        "email": email,
        "password": password
    }
    auth_response = requests.post(auth_url, json=auth_data)
    
    if auth_response.status_code != 200:
        raise Exception(f"Failed to get token: {auth_response.status_code} - {auth_response.text}")
    
    token = auth_response.json().get('token')
    
    # 发送消息
    chat_url = f"{base_url}/api/chat/completions"
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }
    chat_data = {
        "stream": True,
        "model": model,
        "messages": [
            {
                "role": "user",
                "content": message
            }
        ],
        "params": {},
        "features": {
            "image_generation": False,
            "code_interpreter": False,
            "web_search": False
        },
        "variables": {
            "{{USER_NAME}}": "lyh",
            "{{USER_LOCATION}}": "Unknown",
            "{{CURRENT_DATETIME}}": "2025-05-23 13:25:01",
            "{{CURRENT_DATE}}": "2025-05-23",
            "{{CURRENT_TIME}}": "13:25:01",
            "{{CURRENT_WEEKDAY}}": "Friday",
            "{{CURRENT_TIMEZONE}}": "Asia/Shanghai",
            "{{USER_LANGUAGE}}": "zh-CN"
        },
        "chat_id": "local",
    }
    
    with requests.post(chat_url, headers=headers, json=chat_data, stream=True) as response:
        if response.status_code != 200:
            raise Exception(f"Failed to send message: {response.status_code} - {response.text}")
        
        for line in response.iter_lines():
            if line:
                decoded_line = line.decode('utf-8')
                if decoded_line.startswith("data:"):
                    data = decoded_line[5:]
                    try:
                        json_data = json.loads(data)
                        print(json_data)
                    except json.JSONDecodeError:
                        print(f"Failed to decode JSON: {decoded_line}")

# Example usage
email = "374656045@qq.com"
password = "hrdhreth1"
message = "你好"

completions(email, password, message)

{'id': 'endpoint_common_76151', 'object': 'chat.completion.chunk', 'created': 1749521383, 'model': 'qwen72', 'choices': [{'index': 0, 'delta': {'role': 'assistant', 'content': '你好'}, 'finish_reason': None}]}
{'id': 'endpoint_common_76151', 'object': 'chat.completion.chunk', 'created': 1749521383, 'model': 'qwen72', 'choices': [{'index': 0, 'delta': {'role': 'assistant', 'content': '！'}, 'finish_reason': None}]}
{'id': 'endpoint_common_76151', 'object': 'chat.completion.chunk', 'created': 1749521383, 'model': 'qwen72', 'choices': [{'index': 0, 'delta': {'role': 'assistant', 'content': '有什么'}, 'finish_reason': None}]}
{'id': 'endpoint_common_76151', 'object': 'chat.completion.chunk', 'created': 1749521383, 'model': 'qwen72', 'choices': [{'index': 0, 'delta': {'role': 'assistant', 'content': '可以帮助'}, 'finish_reason': None}]}
{'id': 'endpoint_common_76151', 'object': 'chat.completion.chunk', 'created': 1749521383, 'model': 'qwen72', 'choices': [{'index': 0, 'delta': {'role': 'assistant', '

In [24]:
from openai import OpenAI
client = OpenAI(api_key="sk-a3677832c1654c2aa5b104e01a04b601", base_url="http://71.2.255.46:28080/")
response = client.chat.completions.create(
                model="qwen72",
                messages= [
            {
                "role": "user",
                "content": message
            }
        ],
                stream=False
            )
print(response)

APIStatusError: Error code: 405 - {'detail': 'Method Not Allowed'}